<a href="https://colab.research.google.com/github/julianamo93/CP1_2TDSZ-/blob/main/CP1_2TDSZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Disruptive Architectures IOT, IOB and Generative AI**
- Juliana Moreira - RM: 554113
- Kevin Nobre - RM: 552590

### **CheckPoint 01 - 2TDSZ**

*   Dataset - HouseHold_Power
*   Descrição: Medição de consumo de energia elétrica em uma casa com taxa de amostragem de um minuto ao longo de um período de quase 04 anos. Diferentes grandezas elétricas e alguns valores de submedição estão
disponíveis.
*   Valores ausentes: Sim
*   Tamanho do dataset: 133MB
*   Objetivo: Regressão/Clustering

Importação de Bibliotecas para realização do CheckPoint 01. Bibliotecas padrão para análise e desenvolvimento de DL/ML.

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

## Importação do Dados

Importando os dados do dataset HouseHold Power.


In [3]:
df = pd.read_csv('/content/household_power_consumption.txt', sep=';')
df.head()

<ipython-input-3-8b38190bc575>:1: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/household_power_consumption.txt', sep=';')


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


## Análise Exploratória

Iniciando a análise exploratória (EDA), com os dados do dataset.

In [23]:
df.shape

(2049280, 9)

In [5]:
# df.describe() -- não utilizamos o describe() porque não temos dados numéricos,
# então vamos transformar os dados em numéricos para poder usar o describe()

In [7]:
100 * (25979 / 2075259)

1.2518437457686005

## Convertendo as Colunas para valores

In [28]:
# Converter as colunas 'object' para 'float64'
df = df.apply(pd.to_numeric, errors='coerce')
df = df.replace('?', np.nan)
df = df.astype('float64')

In [29]:
# salvando um novo dataframe eliminando as células vazias
df1 =  df.drop(columns=['Date', 'Time'])

In [24]:
# verificando se existem células vazias e somando os registros encontrados...
# df1.isna().sum()

In [30]:
df1.shape

(2049280, 7)

In [31]:
# Verificando se existem células vazias e somando os registros encontrados
df1.isna().sum()

,0
Global_active_power,0
Global_reactive_power,0
Voltage,0
Global_intensity,0
Sub_metering_1,0
Sub_metering_2,0
Sub_metering_3,0


In [32]:
print(df1.shape)  # Verificar o número de linhas e colunas
print(df1.head())

(2049280, 7)
   Global_active_power  Global_reactive_power  Voltage  Global_intensity  \
0                4.216                  0.418   234.84              18.4   
1                5.360                  0.436   233.63              23.0   
2                5.374                  0.498   233.29              23.0   
3                5.388                  0.502   233.74              23.0   
4                3.666                  0.528   235.68              15.8   

   Sub_metering_1  Sub_metering_2  Sub_metering_3  
0             0.0             1.0            17.0  
1             0.0             1.0            16.0  
2             0.0             2.0            17.0  
3             0.0             1.0            17.0  
4             0.0             1.0            17.0  


## Geração da Amostra de 5% dos Dados

*   Utilizando sample, que preserva nossas colunas e pega apenas a porcentagem escolhida.

In [33]:
# df2 ---> amostra de 5%
df2 = df1.sample(frac=0.05, random_state=10)

In [34]:
df2.shape

(102464, 7)

## Aplicação do Principal Component Analysis (PCA) ---> Redução de Dimensionalidade

In [35]:
print(df2.shape)  # Isso vai mostrar a forma do DataFrame
print(df2.head())

(102464, 7)
         Global_active_power  Global_reactive_power  Voltage  \
1766557                0.364                  0.118   242.06   
750095                 1.238                  0.000   242.02   
1110516                0.840                  0.206   241.48   
775087                 0.486                  0.000   242.44   
598955                 1.380                  0.000   244.51   

         Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
1766557               1.6             0.0             0.0             0.0  
750095                5.0             0.0             0.0            19.0  
1110516               3.6             0.0             1.0             0.0  
775087                2.0             0.0             0.0             1.0  
598955                5.6             0.0             0.0            18.0  


In [36]:
# Número original de atributos = 7
# Número de Componentes após a redução =
n = 3
pca = PCA(n_components=n)

In [37]:
X_pca = pca.fit_transform(df2)
X_pca.shape

(102464, 3)

In [38]:
X_pca.shape

(102464, 3)

In [39]:
# O método "explained_variance_ratio_" ajuda a
# porcentagem de variância dos dados que consegue ser explicada por cada um dos componentes selecionados
pca.explained_variance_ratio_


array([0.49372793, 0.23576995, 0.19283609])

In [40]:
total_variancia = pca.explained_variance_ratio_.sum()
print(f'O modelo PCA com {n} variáveis explica{100*(total_variancia): .2f} % dos dados de entrada.')

O modelo PCA com 3 variáveis explica 92.23 % dos dados de entrada.


## **Agrupamento (Clustering) com K-Means**

*   Global_active_power e Global_reactive_power - escolhidos por medirem o consumo total e apotência reativa, que são características fundamentais para entender o padrão de consumo.
*   Voltage - Para ajudar a identificar variações no consumo associadas a diferentes níveis de tensão.
*   Global_intensity - relaciona-se diretamente à carga total, importante para o perfil de consumo.
*   Sub_metering_1, Sb_metering_2 e Sub_metering_3 - representam o consumo em diferentes áreas, oferecendo uma visão detalhada do padrão de uso.

In [41]:
# Selecionar as colunas relevantes
cols_to_use = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
X = df2[cols_to_use]

# Verificar os dados
print(X.head())

         Global_active_power  Global_reactive_power  Voltage  \
1766557                0.364                  0.118   242.06   
750095                 1.238                  0.000   242.02   
1110516                0.840                  0.206   241.48   
775087                 0.486                  0.000   242.44   
598955                 1.380                  0.000   244.51   

         Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
1766557               1.6             0.0             0.0             0.0  
750095                5.0             0.0             0.0            19.0  
1110516               3.6             0.0             1.0             0.0  
775087                2.0             0.0             0.0             1.0  
598955                5.6             0.0             0.0            18.0  


## **Aplicando o Algoritmo K-Means**

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# definindo a faixa de valores de K  serem testados
k_values = range(1, 11) # teste de 1 a 10 clusters
inertia= []
silhouette_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(X)

    # adicionando inercia para a curva do cotovelo
    inertia.append(kmeans.inertia_)

    # Calculando o silhouette score se houver pelo menos 2 clusters
    if k > 1 and len(set(kmeans.labels_)) > 1:  # Verifica se há múltiplos clusters
        silhouette_scores.append(silhouette_score(X, kmeans.labels_))
    else:
        silhouette_scores.append(None)

# Plotando a Curva de Elbow
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(k_values, inertia, marker='o')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Inércia')
plt.title('Curva de Elbow')

# Plotando o Silhouette Score
plt.subplot(1, 2, 2)
plt.plot(k_values, silhouette_scores, marker='o')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score')

plt.tight_layout()
plt.show()

Após observar a Curva de Elbow, encontramos um ponto onde a redução da Inertia começa a desacelerar, e é esse ponto que encontramos o melhor número de Clusters.

## **Visualizando os Clusters**

*   Agora, vamos visualizar os clusters utilizando gráficos de dispersão.

In [ ]:
import seaborn as sns

# Escolhendo o melhor número de Clusters
k = 3
kmeans = KMeans(n_clusters=k, random_state=10)
clusters = kmeans.fit_predict(X)
df2['Cluster'] = clusters

# Plotando gráficos de dispersão
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df2, x='Global_active_power', y='Global_reactive_power', hue='Cluster', palette='Set1')
plt.title('Clusters com K-means')
plt.xlabel('Global_active_power')
plt.ylabel('Global_reactive_power')
plt.legend()
plt.show()